# Textklassifikation

Die Klassifikation von Texten ist einer der zentralen Anwendungsfälle von NLP. 
Er begegnet uns ständig, etwa in Form eines Spam-Filters in unserem Email-Client oder bei der Intent-Erkennung bei einem Sprachassistenten.

Gleichzeitig eignet sich die Textklassifikation als Beispiel dafür, welche Fortschritte in den letzten Jahren im Bereich NLP gemacht wurden,
etwa bei der Vektorisierung mit *Word2Vec* oder dem Einsatz von transformer-basierten Modellen wie *BERT*.

Wir werden als Beispiel die Daten aus dem Wettbewerb *GermEval 2018* (s. `../data/GermEval-2018`) verwenden.

## Aufgabe 1: Lesen & Aufbereiten der Texte

Um die Texte zu klassifizieren, müssen wir die Trainingsdaten lesen und aufbereiten. Schauen Sie sich dazu zunächst die ersten 10 Trainingsdatensätze an (s. folgende Zelle).

In [3]:
!head -10 ../data/GermEval-2018/germeval2018.training.txt

@corinnamilborn Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?	OTHER	OTHER
@Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.	OTHER	OTHER
@ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️	OTHER	OTHER
@dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!	OTHER	OTHER
@spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung.	OFFENSE	INSULT
@Dirki_M Ja, aber wo widersprechen die Zahlen denn denen, die im von uns verlinkten Artikel stehen? In unserem Tweet geht es rein um subs. Geschützte. 2017 ist der gesamte Familiennachzug im Vergleich zu 2016 - die Zahlen, die Hr. Brandner bemüht - übrigens leicht rückläufig gewesen.	OTHER	OTHER
@milenahanm 33 bis 45 habe ich noch gar

### Aufgabe 1.1: Auffälligkeiten in den Texten

Was fällt Ihnen an den Texten auf? Welche Bestandteile sollten ggf. aus den Trainingsdaten entfernt werden, da sie das Training verfälschen würden?
Welche anderen Bereinigungen könnten sinnvoll sein?

> 💡 **Tipp:** 
> Fragen Sie gerne auch ChatGPT, was man bei der Bereinigung der Texte tun sollte. 

### Aufgabe 1.2: Lesen der Trainings- und Testdatensätze

Die Trainingsdaten liegen als Textdatei mit drei durch `TAB` getrennten Spalten vor.

| TEXT | COARSE | FINE |
| ---- | ------ | ---- |
| @corinnamilborn Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?  | OTHER | OTHER |
| @Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir. | OTHER | OTHER |
| @ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️ | OTHER | OTHER |
| @dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..! | OTHER | OTHER |
| @spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung. | OFFENSE | INSULT | 

Lesen Sie die Daten und wandeln Sie ihn in eine Liste von `namedtuple` mit den Feldern `text`, `coarse_label` und `fine_label` um.

> 💡 **Tipp:** 
> ChatGPT kann Ihnen beim Code sicher helfen

In [1]:
from collections import namedtuple
import csv

# Definiert ein namedtuple für die Daten
TweetData = namedtuple('TweetData', ['text', 'coarse_label', 'fine_label'])

def read_data(filename):
    data = []
    with open(filename, 'r', encoding='utf-8') as file:
        # Verwendet csv.reader zum Lesen der Datei, mit einem Tab als Trennzeichen
        reader = csv.reader(file, delimiter='\t')
        next(reader)  # Überspringt die Kopfzeile
        for row in reader:
            if len(row) == 3:  # Stellt sicher, dass die Zeile die erwartete Anzahl von Elementen enthält
                tweet = TweetData(text=row[0], coarse_label=row[1], fine_label=row[2])
                data.append(tweet)
    return data

# Verwenden Sie die Funktion und geben Sie die Daten aus
filename = "../data/GermEval-2018/germeval2018.training.txt"
tweet_data = read_data(filename)

# Zum Testen, geben wir die ersten paar Datensätze aus
for tweet in tweet_data[:5]:  # Ändern Sie 5, um mehr oder weniger Datensätze auszugeben
    print(tweet)

TweetData(text='@Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.', coarse_label='OTHER', fine_label='OTHER')
TweetData(text='@ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️', coarse_label='OTHER', fine_label='OTHER')
TweetData(text='@dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!', coarse_label='OTHER', fine_label='OTHER')
TweetData(text='@spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung.', coarse_label='OFFENSE', fine_label='INSULT')
TweetData(text='@Dirki_M Ja, aber wo widersprechen die Zahlen denn denen, die im von uns verlinkten Artikel stehen? In unserem Tweet geht es rein um subs. Geschützte. 2017 ist der gesamte Familiennachzug im Vergleich zu 2016 - die Zahlen, die Hr. Brandner bemüht - ü

### Aufgabe 1.3: Bereinigung der Trainingsdatensätze

Setzen Sie **mindestens** die folgenden Textbereinigungen um:

- Entfernen von *Twitter-Handles* wie `@corinnamilborn` (Warum ist das sinnvoll?)
- Entfernen des Hashtag-Zeichens `#` (Warum ist das sinnvoll?)

Zusätzlich können Sie mit anderen Bereinigungen experimentieren (Entfernung Groß- und Kleinschreibung, Stemming, ...). Im Deutschen bringt das allerdings nicht so viel.

In [2]:
import re
from collections import namedtuple
import csv

# Definiert ein namedtuple für die Daten
TweetData = namedtuple('TweetData', ['text', 'coarse_label', 'fine_label'])

def clean_text(text):
    # Entfernt Twitter-Handles
    text = re.sub(r'@\w+', '', text)
    # Entfernt das Hashtag-Zeichen, behält aber das Wort bei
    text = text.replace('#', '')
    # Hier könnten weitere Bereinigungsschritte hinzugefügt werden, wie z.B. das Entfernen von URLs, Zahlen etc.
    text = text.lower()  # Optional: Konvertierung in Kleinbuchstaben
    return text.strip()  # Entfernt führende und abschließende Leerzeichen

def read_data(filename):
    data = []
    with open(filename, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')
        next(reader)  # Überspringt die Kopfzeile
        for row in reader:
            if len(row) == 3:
                cleaned_text = clean_text(row[0])
                tweet = TweetData(text=cleaned_text, coarse_label=row[1], fine_label=row[2])
                data.append(tweet)
    return data

# Liest Daten und bereinigt sie
filename = "../data/GermEval-2018/germeval2018.training.txt"
tweet_data = read_data(filename)

# Gibt die ersten paar Datensätze aus, zum Testen
for tweet in tweet_data[:5]:
    print(tweet)

TweetData(text='sie haben ja auch recht. unser tweet war etwas missverständlich. dass das bverfg sachleistungen nicht ausschließt, kritisieren wir.', coarse_label='OTHER', fine_label='OTHER')
TweetData(text='fröhlicher gruß aus der schönsten stadt der welt theo ⚓️', coarse_label='OTHER', fine_label='OTHER')
TweetData(text='amis hätten alles und jeden gewählt...nur hillary wollten sie nicht und eine fortsetzung von obama-politik erst recht nicht..!', coarse_label='OTHER', fine_label='OTHER')
TweetData(text='kein verläßlicher verhandlungspartner. nachkarteln nach den sondierzngsgesprächen - schickt diese stümper spd in die versenkung.', coarse_label='OFFENSE', fine_label='INSULT')
TweetData(text='ja, aber wo widersprechen die zahlen denn denen, die im von uns verlinkten artikel stehen? in unserem tweet geht es rein um subs. geschützte. 2017 ist der gesamte familiennachzug im vergleich zu 2016 - die zahlen, die hr. brandner bemüht - übrigens leicht rückläufig gewesen.', coarse_label='OTHE

### Aufgabe 1.4: Training eines Naive-Bayes-Klassifikators

Mithilfe von `scikit-learn` ist es sehr einfach, eine Naive-Bayes-Klassifikator zu trainieren. Sie benötigen im einfachsten Fall nur den
[`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) zur Vektorisierung und 
[`MultinomialNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) für die Klassifikation zu einer Pipeline zu verbinden.

> 💡 **Tipp:** 
> Auch hier kann ChatGPT Ihnen beim Code helfen ...

Trainieren Sie den Klassifikator auf den Trainingsdaten und messen Sie die Accuracy auf den Testdaten. Wie gut ist Ihr Ergebnis? Vergleichen Sie Ihr Ergebnis mit den [Ergebnissen des GermEval-2018](../data/GermEval-2018/results.pdf).

2